# SuSiE RSS

Bayesian sum of single-effect (SuSiE) linear regression using z scores

After applying LD_Clumping.ipynb and Region_Extraction.ipynb to select regions that overlap between traits, the current pipeline focuses on SuSiE to do fine mapping of those regions to see if theres something of interest.

To run this notebook follow the example:

```
sos run ~/project/UKBB_GWAS_dev/workflow/SuSiE_test.ipynb \
    --cwd /gpfs/gibbs/pi/dewan/data/UKBiobank/results/fine_mapping/f3393_hearing_aid \
    --region_dir /gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid \
    --region_file /gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid/regions.txt \
    --sumstats_path /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/*.snp_stats.gz \
    --N 230411 \
    --container_lmm /home/dc2325/scratch60/lmm_v_1_4.sif \
    --container_marp /gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif -s build
```

In [1]:
[global]
# Path to region extraction files
parameter: region_dir = path
#The region file after LD clumping
parameter: region_file = path
parameter: sumstats_path = path
fail_if(not region_file.is_file(), msg = 'Cannot find regions to fine map. Please specify them using ``--region-file`` option.')
# Load all regions of interest. Each item in the list will be a region: (chr, start, end)
regions = [x.strip() for x in open(region_file).readlines()]
regions = [x.replace(' ', '_' ) for x in regions]
#The directory for output files
parameter: cwd = path
## The container with the lmm/marp software. Can be either a dockerhub image or a singularity `sif` file.
parameter: container_lmm = 'statisticalgenetics/lmm:1.4'
parameter: container_marp = 'gaow/marp'
# Specific number of threads to use
parameter: numThreads = 2

In [1]:
[default_1]
parameter: N = int
input: [(f"{region_dir}/{x}/{sumstats_path:bn}_{x}.sumstats.gz", f"{region_dir}/{x}/{sumstats_path:bn}_{x}.sample_ld.gz") for x in regions], group_by = 2
output: [f'{cwd}/{x}.{sumstats_path:bnn}.SuSiE_RSS.rds' for x in regions], group_by=1
task: trunk_workers = 1, trunk_size = job_size, walltime = '12h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: container=container_lmm, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    sumstat = read.csv(${_input[0]:r}, sep = '\t', header=T,stringsAsFactors=F)
    ld = read.csv(${_input[1]:r}, sep = '\t', header=T, stringsAsFactors=F)
    ld = as.matrix(ld[,-1])
    res = susieR::susie_rss(as.double(sumstat$Z), ld, z_ld_weight = 1/${N}, 
                            L = 10,
                            estimate_residual_variance = TRUE, check_R=F,
                            estimate_prior_variance = TRUE, check_z = F)
    res$pos = as.integer(sumstat$POS)
    res$z = as.double(sumstat$Z)
    res$p = as.double(sumstat$P)
    res$var_names = sumstat$SNP
    saveRDS(res, ${_output:r})

In [1]:
[default_2]
output: pip_plot = f"{cwd}/{_input:bn}.png", group_by=1
task: trunk_workers = 1, trunk_size = job_size, walltime = '12h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: container=container_lmm, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    res = readRDS(${_input:r})
    png(${_output:r}, width = 8, height=5, unit='in', res=300)
    susieR::susie_plot(res, y= "PIP", pos=list(attr='pos',start=res$pos[1],end=res$pos[length(res$pos)]))
    dev.off()

In [1]:
[default_3]
sep = '\n\n---\n'
input: group_by = 'all'
output: analysis_summary = f'{cwd}/{sumstats_path:bnn}.analysis_summary.md'
bash: container=container_lmm, expand = "${ }"
    echo '''---
    theme: base-theme
    style: |
     p {
       font-size: 24px;
       height: 900px;
       margin-top:1cm;
      }
      img {
        height: 70%;
        display: block;
        margin-left: auto;
        margin-right: auto;
      }
      body {
       margin-top: auto;
       margin-bottom: auto;
       font-family: verdana;
      }
    ---    
    ''' > ${_output}
    set -e
    for i in ${_input:bn}; do
    echo -e  "#\n\n Susie RSS $i \n" >> ${_output}
    echo -e "![]($i.png)${sep}" >> ${_output}
    done

In [ ]:
# Generate analysis report: HTML file, and optionally PPTX file
[default_4]
output: f"{_input['analysis_summary']:n}.html"
sh: container=container_marp, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:a} \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:an}.pptx \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files